

# Predicting Customer Satisfaction on Rent the Runway

##  I. Data Cleaning for Customer Data 
### Katrin Ayrapetov


<font style="font-size: 2rem; color: blue">


 
</font>

In [1]:
#Import the necessary libraries. 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

In [2]:
#Import the Data Set 

df1 =  pd.read_csv('../Data/Raw_Data_Rent_the_Runway(1).csv')
df2 =  pd.read_csv('../Data/Raw_Data_Rent_the_Runway(2).csv')
df3 =  pd.read_csv('../Data/Raw_Data_Rent_the_Runway(3).csv')
df4 =  pd.read_csv('../Data/Raw_Data_Rent_the_Runway(4).csv')
df5 =  pd.read_csv('../Data/Raw_Data_Rent_the_Runway(5).csv')
df6 =  pd.read_csv('../Data/Raw_Data_Rent_the_Runway(6).csv')
df7 =  pd.read_csv('../Data/Raw_Data_Rent_the_Runway(7).csv')
df8 =  pd.read_csv('../Data/Raw_Data_Rent_the_Runway(8).csv')
df9 =  pd.read_csv('../Data/Raw_Data_Rent_the_Runway(9).csv')
df10 =  pd.read_csv('../Data/Raw_Data_Rent_the_Runway(10).csv')
df11 =  pd.read_csv('../Data/Raw_Data_Rent_the_Runway(11).csv')

In [3]:
#Join all the data sets into one data set 
df = pd.concat([df1, df2,df3,df4,df5,df6,df7,df8,df9,df10,df11])
df = df.reset_index(drop=True)

In [4]:
#The data set has 24 features and 208,474 observations. 
df.shape

(208474, 24)

In [5]:
#Create a checklist which will be used to make sure the cleaning is fone correctly. 
check_list=[1,45,200,1100,6565,7500]
check_list

[1, 45, 200, 1100, 6565, 7500]

In [6]:
#Rename the columns 
df.columns = ["title_of_review",
              "image","Nickname","Type_of_Customer",
              "Size","Overall_fit","Rented_for","Size_usually_worn",
              "Height","Age","Label6","Bust_size",
              "Label8","Body_type","Weight","Rating",
              "Date","Review_body","Brand","Dress_Description",
              "Retail_price", "Rent_price","Product_details","Number_of_reviews"]


In [7]:
#Drop columns that are not using 
df.drop(columns = ["title_of_review","image","Nickname","Review_body"],inplace=True)

In [8]:
#Change the date when the garment was rented to just the season.
for i in range(df.shape[0]):
    if "JANUARY" in df["Date"][i] or "DECEMBER" in df["Date"][i] or "FEBRUARY" in df["Date"][i]:
        df.at[i, "Date"] = "winter"
    if "AUGUST" in df["Date"][i] or "JUNE" in df["Date"][i] or "JULY" in df["Date"][i]:
        df.at[i, "Date"] = "summer"
    if "SEPTEMBER" in df["Date"][i] or "NOVEMBER" in df["Date"][i] or "OCTOBER" in df["Date"][i]:
        df.at[i, "Date"] = "fall"
    if "MAY" in df["Date"][i] or "APRIL" in df["Date"][i] or "MARCH" in df["Date"][i]:
        df.at[i, "Date"] = "spring"

In [9]:
#Fill in the nulls in Type_of_customer feature with "unknown"
df['Type_of_Customer'].fillna("unknown", inplace=True)

In [10]:
#This is the breakdown for type_of_customer feature. 
df.Type_of_Customer.value_counts()

TOP CONTRIBUTOR    163504
unknown             44970
Name: Type_of_Customer, dtype: int64

In [11]:
#Convert the rating that the customer left to just an integer
print(f"The data frame has {df.shape[0]} observations. There are {df.Rating.isna().sum()} nulls for the Rating feature. ")
print(df.Rating.value_counts())
#Get rid of unnecessary characters in some entries. 
df["Rating"]=df["Rating"].replace(" out of 5 stars","", regex=True)
df["Rating"]=df["Rating"].replace("\(","", regex=True)
df["Rating"]=df["Rating"].replace("\)","", regex=True)
df["Rating"]=df["Rating"].astype("int")
print(df.Rating.value_counts())

The data frame has 208474 observations. There are 0 nulls for the Rating feature. 
(5 out of 5 stars)    125250
(4 out of 5 stars)     47480
(3 out of 5 stars)     20194
(2 out of 5 stars)      8681
(1 out of 5 stars)      6869
Name: Rating, dtype: int64
5    125250
4     47480
3     20194
2      8681
1      6869
Name: Rating, dtype: int64


In [12]:
#Get rid of observations where there are nulls in ALL 5 customer measurement features. 
df.dropna(how="all",subset=['Age','Weight','Body_type','Bust_size','Height'], inplace=True)
df = df.reset_index()
df.drop(columns=["index"],inplace=True)

In [13]:
#Fill in the null values in columns Size Usually Worn, Age, Weight, Body_type, Bust_size and Height as "unknown" for now
df['Rented_for'].fillna("unknown", inplace=True)
df['Size_usually_worn'].fillna("unknown", inplace=True)
df['Age'].fillna("unknown", inplace=True)
df['Weight'].fillna("unknown", inplace=True)
df['Body_type'].fillna("unknown", inplace=True)
df['Bust_size'].fillna("unknown", inplace=True)
df['Height'].fillna("unknown", inplace=True)

In [14]:
#Weight in Body Type fix rows 
weight_in_body_type=[]
for i in range(df.shape[0]):
    if "LBS" in df["Body_type"][i]:
        weight_in_body_type.append(i)
print(f"There are {len(weight_in_body_type)} observations where the weight is entered in the Body Type column. ")
print("Should match with this number:")
print(df.Label8.value_counts()["Weight: "])

print("Example of one such row BEFORE the transformation:")
print(f"This is the body type entry for row {weight_in_body_type[0]}")
print(df["Body_type"][weight_in_body_type[0]])
print(f"This is the weight type entry for {weight_in_body_type[0]}")
print(df["Weight"][weight_in_body_type[0]])

#Grab all the weights accidently entered into the Body Type column:
#(1) Enter them into the weight column
#(2) Replace the body type observation with "unknown"
for i in weight_in_body_type:
    df.at[i, "Weight"] = df["Body_type"][i]
    df.at[i, "Body_type"] = "unknown"
    
    
print("Example of one such row AFTER the transformation:")
print(f"This is the body type entry for row {weight_in_body_type[0]}")
print(df["Body_type"][weight_in_body_type[0]])
print(f"This is the weight type entry for {weight_in_body_type[0]}")
print(df["Weight"][weight_in_body_type[0]])

There are 12223 observations where the weight is entered in the Body Type column. 
Should match with this number:
12223
Example of one such row BEFORE the transformation:
This is the body type entry for row 13
155LBS
This is the weight type entry for 13
unknown
Example of one such row AFTER the transformation:
This is the body type entry for row 13
unknown
This is the weight type entry for 13
155LBS


In [15]:
#Weight in Bust_Size fix rows 
weight_in_bust_size=[]
for i in range(df.shape[0]):
    if "LBS" in df["Bust_size"][i]:
        weight_in_bust_size.append(i)

print(f"There are {len(weight_in_bust_size)} observations where the weight is entered in the Bust Size column. ")
print("Should match with this number:")
print(df.Label6.value_counts()["Weight: "])

print("Example of one such row before the transformation:")
print(f"This is the bust size  entry for row {weight_in_bust_size[0]}")
print(df["Bust_size"][weight_in_bust_size[0]])
print(f"This is the weight type entry for {weight_in_bust_size[0]}")
print(df["Weight"][weight_in_bust_size[0]])

#Grab all the weights accidently entered into the Bust Size column:
#(1) Enter them into the weight column
#(2) Replace the Bust Size  observation with "unknown"
for i in weight_in_bust_size:
    df.at[i, "Weight"] = df["Bust_size"][i]
    df.at[i, "Bust_size"] = "unknown"
print("Example of one such row AFTER the transformation:")
print(f"This is the bust size  entry for row {weight_in_bust_size[0]}")
print(df["Bust_size"][weight_in_bust_size[0]])
print(f"This is the weight type entry for {weight_in_bust_size[0]}")
print(df["Weight"][weight_in_bust_size[0]])

There are 958 observations where the weight is entered in the Bust Size column. 
Should match with this number:
958
Example of one such row before the transformation:
This is the bust size  entry for row 326
135LBS
This is the weight type entry for 326
unknown
Example of one such row AFTER the transformation:
This is the bust size  entry for row 326
unknown
This is the weight type entry for 326
135LBS


In [16]:
#Weight in Age fix rows 
weight_in_age=[]
for i in range(df.shape[0]):
    if "LBS" in df["Age"][i]:
        weight_in_age.append(i)

print(f"There are {len(weight_in_age)} observations where the weight is entered in the age column. ")

print("Example of one such row BEFORE the transformation:")
print(f"This is the age  entry for row {weight_in_age[0]}")
print(df["Age"][weight_in_age[0]])
print(f"This is the weight entry for {weight_in_age[0]}")
print(df["Weight"][weight_in_age[0]])

#Grab all the weights accidently entered into the Age column:
#(1) Enter them into the weight column
#(2) Replace the Age  observation with "unknown"
for i in weight_in_age:
    df.at[i, "Weight"] = df["Age"][i]
    df.at[i, "Age"] = "unknown"
print("Example of one such row AFTER the transformation:")
print(f"This is the age  entry for row {weight_in_age[0]}")
print(df["Age"][weight_in_age[0]])
print(f"This is the weight entry for {weight_in_age[0]}")
print(df["Weight"][weight_in_age[0]])

There are 63 observations where the weight is entered in the age column. 
Example of one such row BEFORE the transformation:
This is the age  entry for row 6339
155LBS
This is the weight entry for 6339
unknown
Example of one such row AFTER the transformation:
This is the age  entry for row 6339
unknown
This is the weight entry for 6339
155LBS


In [17]:
#Weight in Height fix rows 
weight_in_height=[]
for i in range(df.shape[0]):
    if "LBS" in df["Height"][i]:
        weight_in_height.append(i)
print(f"There are {len(weight_in_height)} observations where the weight is entered in the height column. ")
if len(weight_in_height)>0:
    print(f"There are {len(weight_in_height)} observations where the weight is entered in the height column. ")
    print("Example of one such row BEFORE the transformation:")
    print(f"This is height entry for row {weight_in_height[0]}")
    print(df["Height"][weight_in_height[0]])
    print(f"This is the weight entry for {weight_in_height[0]}")
    print(df["Weight"][weight_in_height[0]])

#Grab all the weights accidently entered into the Height column:
#(1) Enter them into the weight column
#(2) Replace the Height  observation with "unknown"
for i in weight_in_height:
    df.at[i, "Weight"] = df["Height"][i]
    df.at[i, "Height"] = "unknown"
print("Example of one such row before the transformation:")
print(f"This is height entry for row {weight_in_height[0]}")
print(df["Height"][weight_in_height[0]])
print(f"This is the weight entry for {weight_in_height[0]}")
print(df["Weight"][weight_in_height[0]])

There are 7 observations where the weight is entered in the height column. 
There are 7 observations where the weight is entered in the height column. 
Example of one such row BEFORE the transformation:
This is height entry for row 31076
130LBS
This is the weight entry for 31076
unknown
Example of one such row before the transformation:
This is height entry for row 31076
unknown
This is the weight entry for 31076
130LBS


In [18]:
#Strip the weights off the "LBS" and clean up weights entered two or three times. 
print("Before:")
for i in check_list:
    print(df["Weight"][i])
for i in range(df.shape[0]):
    if df["Weight"][i][0]=="1" or df["Weight"][i][0]=="2":
        df.at[i, "Weight"]=df["Weight"][i][:3]
    elif df["Weight"][i][0]=="9" or df["Weight"][i][0]=="8":
        df.at[i, "Weight"]=df["Weight"][i][:2]
    elif df["Weight"][i][0]=="3":
        df.at[i, "Weight"] = "unknown"

df["Weight"]=df["Weight"].replace("l","0", regex=True).replace("L","0", regex=True)

for i in range(df.shape[0]):
    if "b" in df["Weight"][i].lower():
        df.at[i, "Weight"] = "unknown"
print("After:")
for i in check_list:
    print(df["Weight"][i])

Before:
unknown
145LBS
180LBS
unknown
125LBS
115LBS
After:
unknown
145
180
unknown
125
115


In [19]:
list_of_body_types = ['athletic','hourglass','pear','apple','fullbust','straightnarrow','petite','ATHLETIC',
 'HOURGLASS',
 'PEAR',
 'APPLE',
 'FULLBUST',
 'STRAIGHTNARROW',
 'PETITE']

In [20]:
#look at the rows where the body type has been entered into the bust_size 
body_type_in_bust_size=[]
for i in range(df.shape[0]):
    if df["Label6"][i]=="Body type: ":
        body_type_in_bust_size.append(i)
print(f"There are {len(body_type_in_bust_size)} observations where the body_type is entered in the Bust_size column. ")
print("Should match with this number:")
print(df.Label6.value_counts()["Body type: "])
print("These are one such row before transformation")
print(f"This is bust size  entry for row {body_type_in_bust_size[5]}")
print(df["Bust_size"][body_type_in_bust_size[5]])
print(f"This is the body type entry for {body_type_in_bust_size[5]}")
print(df["Body_type"][body_type_in_bust_size[5]])

#Grab all the body type accidently entered into the bust size  column:
#(1) Enter them into the body type column
#(2) Replace the bust size  observation with "unknown"
for i in body_type_in_bust_size:
    df.at[i, "Body_type"] = df["Bust_size"][i].lower().replace(" & ","").replace(" ","")
    df.at[i, "Bust_size"]="unknown"
print("These are one such row AFTER  transformation")
print(f"This is bust size  entry for row {body_type_in_bust_size[5]}")
print(df["Bust_size"][body_type_in_bust_size[5]])
print(f"This is the body type entry for {body_type_in_bust_size[5]}")
print(df["Body_type"][body_type_in_bust_size[5]])

There are 18232 observations where the body_type is entered in the Bust_size column. 
Should match with this number:
18232
These are one such row before transformation
This is bust size  entry for row 29
HOURGLASS
This is the body type entry for 29
unknown
These are one such row AFTER  transformation
This is bust size  entry for row 29
unknown
This is the body type entry for 29
hourglass


In [21]:
#Get rid of the + symbol in the some of the bust sizes. 
df["Bust_size"]=df["Bust_size"].replace("\+","", regex=True)

In [22]:
#look at the rows where the body type has been entered into the Age 
body_type_in_age=[]
for i in range(df.shape[0]):
    if df["Age"][i].lower().replace(" & ","").replace(" ","") in list_of_body_types:
        body_type_in_age.append(i)
        
print(f"There are {len(body_type_in_age)} observations where the body_type is entered in the Bust_size column. ")

print("These are one such row before transformation")
print(f"This is Age entry for row {body_type_in_age[-1]}")
print(df["Age"][body_type_in_age[-1]])
print(f"This is the body type entry for {body_type_in_age[-1]}")
print(df["Body_type"][body_type_in_age[-1]])

#Grab all the body type accidently entered into the Age  column:
#(1) Enter them into the body type column
#(2) Replace the Age  observation with "unknown"
for i in body_type_in_age:
    df.at[i, "Body_type"] = df["Age"][i].lower().replace(" & ","").replace(" ","")
    df.at[i, "Age"]="unknown"
    
    
print("These are one such row AFTER transformation")
print(f"This is Age entry for row {body_type_in_age[-1]}")
print(df["Age"][body_type_in_age[-1]])
print(f"This is the body type entry for {body_type_in_age[-1]}")
print(df["Body_type"][body_type_in_age[-1]])

There are 184 observations where the body_type is entered in the Bust_size column. 
These are one such row before transformation
This is Age entry for row 202175
ATHLETIC
This is the body type entry for 202175
unknown
These are one such row AFTER transformation
This is Age entry for row 202175
unknown
This is the body type entry for 202175
athletic


In [23]:
#look at the rows where the body type has been entered into the Height 
body_type_in_height=[]
for i in range(df.shape[0]):
    if df["Height"][i].lower().replace(" & ","").replace(" ","") in list_of_body_types:
        body_type_in_height.append(i)
print(f"There are {len(body_type_in_height)} observations where the body_type is entered in the Height column. ")
if len(body_type_in_height)>0:
    print("These are one such row before transformation")
    print(f"This is height entry for row {body_type_in_height[0]}")
    print(df["Height"][body_type_in_height[0]])
    print(f"This is the body type entry for {body_type_in_height[0]}")
    print(df["Body_type"][body_type_in_height[0]])
    
    #Grab all the body type accidently entered into the Height  column:
    #(1) Enter them into the body type column
    #(2) Replace theHeight observation with "unknown"
    for i in body_type_in_height:
        df.at[i, "Body_type"] = df["Height"][i].lower().replace(" & ","").replace(" ","")
        df.at[i, "Height"]="unknown"


    print("These are one such row AFTER transformation")
    print(f"This is height entry for row {body_type_in_height[0]}")
    print(df["Height"][body_type_in_height[0]])
    print(f"This is the body type entry for {body_type_in_height[0]}")
    print(df["Body_type"][body_type_in_height[0]])


There are 7 observations where the body_type is entered in the Height column. 
These are one such row before transformation
This is height entry for row 14398
HOURGLASS
This is the body type entry for 14398
unknown
These are one such row AFTER transformation
This is height entry for row 14398
unknown
This is the body type entry for 14398
hourglass


In [24]:
#Clean up the body types 
for i in range(df.shape[0]):
    df.at[i, "Body_type"] = df["Body_type"][i].lower().replace(" & ","").replace(" ","")

In [25]:
#Final check of Body Types. Should just be body types here. 
df.Body_type.value_counts()

hourglass         60310
athletic          44365
pear              23000
petite            20598
unknown           16623
fullbust          16572
straightnarrow    13833
apple              7082
Name: Body_type, dtype: int64

In [26]:
#Bust_size_in_Age_Column 
bust_size_in_age=[]
pattern = "^.*[0-9]{2}[a-zA-Z]{1}.*$"
for i in range(df.shape[0]):
    if re.match(pattern, df["Age"][i]): 
        bust_size_in_age.append(i)
    
print(f"There are {len(bust_size_in_age)} observations where the bust_size is entered in the age column. ")

print("This is one such observation BEFORE transformation:")
print(f"This is age entry for row {bust_size_in_age[0]}")
print(df["Age"][bust_size_in_age[0]])
print(f"This is the body type entry for {bust_size_in_age[0]}")
print(df["Bust_size"][bust_size_in_age[0]])

#Grab all the bust size  accidently entered into the Age  column:
#(1) Enter them into the bust size  column
#(2) Replace the Age  observation with "unknown"
for i in bust_size_in_age:
    df.at[i, "Bust_size"] = df["Age"][i]
    df.at[i, "Age"] = "unknown"

print("This is one such observation AFTER transformation:")
print(f"This is age entry for row {bust_size_in_age[0]}")
print(df["Age"][bust_size_in_age[0]])
print(f"This is the body type entry for {bust_size_in_age[0]}")
print(df["Bust_size"][bust_size_in_age[0]])

There are 1973 observations where the bust_size is entered in the age column. 
This is one such observation BEFORE transformation:
This is age entry for row 137
34D
This is the body type entry for 137
unknown
This is one such observation AFTER transformation:
This is age entry for row 137
unknown
This is the body type entry for 137
34D


In [27]:
#Bust_size_in_Height_Column 
bust_size_in_height=[]
pattern = "^.*[0-9]{2}[a-zA-Z]{1}.*$"
for i in range(df.shape[0]):
    if re.match(pattern, df["Height"][i]):
        bust_size_in_height.append(i)
print(f"There are {len(bust_size_in_height)} observations where the bust_size is entered in the height column. ")

print("This is one such observation BEFORE transformation:")
print(f"This is height entry for row {bust_size_in_height[0]}")
print(df["Height"][bust_size_in_height[0]])
print(f"This is the bust size  entry for {bust_size_in_height[0]}")
print(df["Bust_size"][bust_size_in_height[0]])

#Grab all the bust size  accidently entered into the Height column:
#(1) Enter them into the bust size  column
#(2) Replace the Height observation with "unknown"
for i in bust_size_in_height:
    df.at[i, "Bust_size"] = df["Height"][i]
    df.at[i, "Height"] = "unknown"

print("This is one such observation AFTER transformation:")
print(f"This is height entry for row {bust_size_in_height[0]}")
print(df["Height"][bust_size_in_height[0]])
print(f"This is the bust size  entry for {bust_size_in_height[0]}")
print(df["Bust_size"][bust_size_in_height[0]])

There are 62 observations where the bust_size is entered in the height column. 
This is one such observation BEFORE transformation:
This is height entry for row 6339
36B
This is the bust size  entry for 6339
unknown
This is one such observation AFTER transformation:
This is height entry for row 6339
unknown
This is the bust size  entry for 6339
36B


In [28]:
#Age entered in Height Columnn
age_in_height = []
for i in range(df.shape[0]):
    if str(df["Height"][1][1]) not in str(df["Height"][i]) and "unknown" not in str(df["Height"][i]):
        age_in_height.append(i)

print(f"There are {len(age_in_height)} observations where the age is entered in the height column. ")   

print("This is one such observation BEFORE transformation:")
print(f"This is age entry for row {age_in_height[0]}")
print(df["Height"][age_in_height[0]])
print(f"This is the height entry for {age_in_height[0]}")
print(df["Age"][age_in_height[0]])

#Grab all the Age accidently entered into the Height column:
#(1) Enter them into the age column
#(2) Replace the Height observation with "unknown"
for i in age_in_height:
    df.at[i, "Age"] = df["Height"][i]
    df.at[i, "Height"] = "unknown"
    
print("This is one such observation AFTER transformation:")
print(f"This is age entry for row {age_in_height[0]}")
print(df["Height"][age_in_height[0]])
print(f"This is the height entry for {age_in_height[0]}")
print(df["Age"][age_in_height[0]])

There are 4895 observations where the age is entered in the height column. 
This is one such observation BEFORE transformation:
This is age entry for row 72
28
This is the height entry for 72
unknown
This is one such observation AFTER transformation:
This is age entry for row 72
unknown
This is the height entry for 72
28


In [29]:
#Height entered in the size usually worn 
height_in_usual_size = []
for i in range(df.shape[0]):
    if str(df["Height"][1][1]) in str(df["Size_usually_worn"][i]):
        height_in_usual_size.append(i)
print(f"There are {len(height_in_usual_size)} observations where the height is entered in the Size_Usually_worn column. ") 

print("This is one such observation BEFORE transformation:")
print(f"This is Size_usually_worn entry for row {height_in_usual_size[0]}")
print(df["Size_usually_worn"][height_in_usual_size[0]])
print(f"This is the height entry for {height_in_usual_size[0]}")
print(df["Height"][height_in_usual_size[0]])


#Grab all the heights accidently entered into the Size Usually Worn column:
#(1) Enter them into the height column
#(2) Replace the Size Usually Worn  observation with "unknown"

for i in height_in_usual_size:
    df.at[i, "Height"] = df["Size_usually_worn"][i]
    df.at[i, "Size_usually_worn"] = "unknown"
    
print("This is one such observation AFTER transformation:")
print(f"This is Size_usually_worn entry for row {height_in_usual_size[0]}")
print(df["Size_usually_worn"][height_in_usual_size[0]])
print(f"This is the height entry for {height_in_usual_size[0]}")
print(df["Height"][height_in_usual_size[0]])

There are 355 observations where the height is entered in the Size_Usually_worn column. 
This is one such observation BEFORE transformation:
This is Size_usually_worn entry for row 826
5' 6"
This is the height entry for 826
unknown
This is one such observation AFTER transformation:
This is Size_usually_worn entry for row 826
unknown
This is the height entry for 826
5' 6"


In [30]:
# "Rented for" observations entered in "Overall Fit"
rented_for_in_overall_fit = []
for i in range(df.shape[0]):
    if "Rented for: " in df["Overall_fit"][i]  and "Rented for: " not in df["Rented_for"][i]:
        rented_for_in_overall_fit.append(i)
print(f"There are {len(rented_for_in_overall_fit)} observations where the Rented_For observation is entered in Overall_fit column. ") 

print("This is one such observation BEFORE transformation:")
print(f"This is Overall Fit  entry for row {rented_for_in_overall_fit[0]}")
print(df["Overall_fit"][rented_for_in_overall_fit[0]])
print(f"This is the Rented for entry for row {rented_for_in_overall_fit[0]}")
print(df["Rented_for"][rented_for_in_overall_fit[0]])

#Grab all the Rented For  entered into the Overall_fit column:
#(1) Enter them into the Rented For column
#(2) Replace the Size Usually Worn  observation with "unknown"

for i in rented_for_in_overall_fit:
    df.at[i, "Rented_for"] = df["Overall_fit"][i]
    df.at[i, "Overall_fit"] = "unknown"
    
print("This is one such observation BEFORE transformation:")
print(f"This is Overall Fit  entry for row {rented_for_in_overall_fit[0]}")
print(df["Overall_fit"][rented_for_in_overall_fit[0]])
print(f"This is the Rented for entry for row {rented_for_in_overall_fit[0]}")
print(df["Rented_for"][rented_for_in_overall_fit[0]])

There are 82651 observations where the Rented_For observation is entered in Overall_fit column. 
This is one such observation BEFORE transformation:
This is Overall Fit  entry for row 62
Rented for: Vacation
This is the Rented for entry for row 62
Usually wears: 6Height: 5' 5"Age: 33Bust size: 34CBody type: Athletic
This is one such observation BEFORE transformation:
This is Overall Fit  entry for row 62
unknown
This is the Rented for entry for row 62
Rented for: Vacation


In [31]:
for i in range(df.shape[0]):
    if "Usually wears: " in df["Overall_fit"][i]:
        df.at[i, "Overall_fit"] = "unknown"

In [32]:
#This should only have the overall fit. 
df.Overall_fit.value_counts()

Overall fit: True to Size    83605
unknown                      82927
Overall fit: Small           20209
Overall fit: Large           15642
Name: Overall_fit, dtype: int64

In [33]:
# "Rented for" observations entered in "Size"
rented_for_in_size = []
for i in range(df.shape[0]):
    if "Rented for: " in df["Size"][i]  and "Rented for: " not in df["Rented_for"][i]:
        rented_for_in_size.append(i)
print(f"There are {len(rented_for_in_size)} observations where the Rented_For observation is entered in Size  column. ") 

print("This is one such observation BEFORE transformation:")
print(f"This is the Size entry for row {rented_for_in_size[0]}")
print(df["Size"][rented_for_in_size[0]])
print(f"This is the Rented for entry for row {rented_for_in_size[0]}")
print(df["Rented_for"][rented_for_in_size[0]])

#Grab all the Rented For  entered into the size column:
#(1) Enter them into the Rented For column
#(2) Replace the Size  observation with "unknown"

for i in rented_for_in_size:
    df.at[i, "Rented_for"] = df["Size"][i]
    df.at[i, "Size"] = "unknown"

print("This is one such observation AFTER transformation:")

print(f"This is the Size entry for row {rented_for_in_size[0]}")
print(df["Size"][rented_for_in_size[0]])
print(f"This is the Rented for entry for row {rented_for_in_size[0]}")
print(df["Rented_for"][rented_for_in_size[0]]) 

There are 276 observations where the Rented_For observation is entered in Size  column. 
This is one such observation BEFORE transformation:
This is the Size entry for row 36757
Rented for: Everyday
This is the Rented for entry for row 36757
unknown
This is one such observation AFTER transformation:
This is the Size entry for row 36757
unknown
This is the Rented for entry for row 36757
Rented for: Everyday


In [34]:
for i in range(df.shape[0]):
    if "Usually wears: " in str(df["Rented_for"][i]):
        df.at[i, "Rented_for"] = "unknown"

In [35]:
df["Rented_for"]=df["Rented_for"].replace("Rented for: ","", regex=True)

In [36]:
df["Rented_for"]=df["Rented_for"].replace("Other","unknown", regex=True)

In [37]:
#This should only have the rented for categories.
df.Rented_for.value_counts()

Work             41377
Wedding          35523
Everyday         30554
Party            30544
unknown          23917
Formal Affair    17709
Vacation         11497
Date             11262
Name: Rented_for, dtype: int64

In [38]:
#Clean up the Size data. Fix typos, European size, etc
df["Size"]=df["Size"].replace("Size worn: ","", regex=True)
df["Size"]=df["Size"].replace("NONE","unknown", regex=True)
df["Size"]=df["Size"].replace("None","unknown", regex=True)
df["Size"]=df["Size"].replace("R","", regex=True)
df["Size"]=df["Size"].replace("P-S","SP", regex=True)
df["Size"]=df["Size"].replace("M-L","L", regex=True)
df["Size"]=df["Size"].replace("8-10","M", regex=True)
df["Size"]=df["Size"].replace("4-6","S", regex=True)
df["Size"]=df["Size"].replace("0-2","XS", regex=True)
df["Size"]=df["Size"].replace("14-16","XL", regex=True)
df["Size"]=df["Size"].replace("XS-S","S", regex=True)
df["Size"]=df["Size"].replace("14/16","XL", regex=True)
df["Size"]=df["Size"].replace("18-20","2XL", regex=True)
df["Size"]=df["Size"].replace("W","", regex=True)
df["Size"]=df["Size"].replace("ONESIZE","OneSize", regex=True)
df["Size"]=df["Size"].replace("12-14","L", regex=True)

In [39]:
#Fix the typo in "Size_usually_worn column"
df["Size_usually_worn"]=df["Size_usually_worn"].replace("GOO4","unknown", regex=True)


In [40]:
#Write a function that converts Heights entered as Feet and inches to just inches. 
def height_to_inches_convert(col):
    
    '''
    function takes in column with feet and inches height format (e.g. 5 ' 5) and returns value in inches
    col: value with initial string format
    '''
    
    # extracting all numbers from the height column so split from special characters
    
    extract = col.str.extractall("(\d+)").reset_index()
    
    # 2 values come out: one for feet and one for inches
    # isolate both using boolean filter: match column indicates first and second value extracted where ft is index 0
    
    ft = extract["match"] == 0
    inches = extract["match"] == 1
    
    # using boolean filter to get ft, inches and then convert both to numeric
    # ft value multiplied by 12 to convert to inches

    ft_conversion = extract[ft].drop(["level_0", "match"], axis=1).reset_index(drop=True).apply(pd.to_numeric) * 12
    inch_conversion = extract[inches].drop(["level_0", "match"], axis=1).reset_index(drop=True).apply(pd.to_numeric)
    
    # add both numbers for total height in inches
    
    return ft_conversion[0] + inch_conversion[0]


In [41]:
#Get rid of observations with unknown value Height Features 
df_unknowns = df[df["Height"]=="unknown"] 
df = df.drop(df_unknowns.index, axis=0) 
df = df.reset_index()


In [42]:
# Convert the Height from feet and inches to just inches. Recast as an integer. 
df["Height"] = height_to_inches_convert(col=df["Height"])
df["Height"]=df["Height"].astype("int")

In [43]:
df.drop(columns="index",inplace=True)
df.drop(columns=["Label6","Label8"],inplace=True)

In [44]:
#Get rid of observations with unknown Ages 
df_unknowns_age = df[df["Age"]=="unknown"] 
df = df.drop(df_unknowns_age.index, axis=0) 
df = df.reset_index()
df.drop(columns="index",inplace=True)
df["Age"]=df["Age"].astype("int")


In [45]:
#Get rid of observations with unknown Weights 
df_unknowns_weight = df[df["Weight"]=="unknown"] 
df = df.drop(df_unknowns_weight.index, axis=0) 
df = df.reset_index()
df.drop(columns="index",inplace=True)
df["Weight"]=df["Weight"].astype("int")

In [46]:
#Drop the height observation where the height is more than 80 inches. 
df.drop(df[df['Height'] > 80].index, inplace = True)
#Drop the age observation where the weight is more than 80 years olds. 
df.drop(df[df['Age'] > 80].index, inplace = True)
df = df.reset_index()
df.drop(columns="index",inplace=True)

In [47]:
df.shape

(161418, 18)

In [48]:
#Body Mass Index, or BMI, is a measurement of body fat based on height and weight.
#It can be calculated using pounds and inches: BMI = (weight in pounds x 703) / (height in inches x height in inches).
df["BMI"] = (df["Weight"]*703)/(df["Height"]*df["Height"])


In [50]:
#check that there are no nulls in customer description features. 
df.isnull().sum()

Type_of_Customer      0
Size                  0
Overall_fit           0
Rented_for            0
Size_usually_worn     0
Height                0
Age                   0
Bust_size             0
Body_type             0
Weight                0
Rating                0
Date                  0
Brand                 0
Dress_Description     0
Retail_price         89
Rent_price           89
Product_details       0
Number_of_reviews     0
BMI                   0
dtype: int64

In [51]:
#Export as an excel file 
df.to_csv('../Data/df_customer_data_cleaned.csv', header=True, index=False)

<font style="font-size: 2rem; color: blue">

3. Clean Garment Description Data 
       
</font>

In [55]:
df["Sleeves"]=df["Sleeves"].replace("elbow-length sleeve","elbow-length sleeves", regex=True)
df["Sleeves"]=df["Sleeves"].replace("three sleeves","three_quarter_sleeves", regex=True)

In [56]:
#Condense the Sleeve Feature to a limited number of features. 
list_of_sleeves = ["sleeveless","long_sleeves","short_sleeves","three_quarter_sleeves","cap_sleeves",
                    "flutter_sleeves","strapless","halter"]
for i in range(df.shape[0]):
    if str(df.at[i, "Sleeves"]) not in list_of_sleeves:
        df.at[i, "Sleeves"] = "other"

In [57]:
#These are the type of Sleeves
df.Sleeves.value_counts()

sleeveless               61444
long_sleeves             36855
short_sleeves            35701
three_quarter_sleeves    12294
cap_sleeves               6888
flutter_sleeves           3760
strapless                 3446
other                      805
halter                     282
Name: Sleeves, dtype: int64

<font style="font-size: 2rem; color: blue">

3. Clean Garment Description Data 
<br> Neckline 
<br>– create a column for type of neckline on a garment. 
</font>

In [58]:
#This extracts the sleeve description from the item description. 
df["Neckline"] = 1
df["Neckline"] = df["Neckline"].astype('object')
for i in range(df.shape[0]):
    for k in reversed(range(len(df["Details"][i]))):
        if "neck" in df["Details"][i][k].lower():
            df.at[i, "Neckline"] = df["Details"][i][k]
            df["Details"][i][k] = "drop"

In [60]:
#Condense the Neckline feature to a limited number of values
list_of_necklines = ["v_neckline","crew_neckline","high_neckline","square_neckline","off_shoulder","shirt_collar_neckline",
                    "sweetheart","boat_neckline","scoop_neckline","turtleneck","mock_neckline","cowl_neckline"]

for i in range(df.shape[0]):
    if str(df.at[i, "Neckline"]) not in list_of_necklines:
        df.at[i, "Neckline"] = "other"

In [61]:
df.Neckline.value_counts()

v_neckline               54187
crew_neckline            40485
high_neckline            11327
square_neckline          10072
off_shoulder              8684
other                     8130
shirt_collar_neckline     5797
sweetheart                5746
boat_neckline             5197
scoop_neckline            5082
turtleneck                3049
mock_neckline             2701
cowl_neckline             1018
Name: Neckline, dtype: int64

<font style="font-size: 2rem; color: blue">

3. Clean Garment Description Data 
<br> Dress Style
<br>– create a column for type of dress of a garment. 
</font>

In [62]:
for i in range(df.shape[0]):
    clean_list=[]
    for k in range(len(df["Details"][i])):
        if "drop" not in df["Details"][i][k]:
            clean_list.append(df["Details"][i][k])
    df.at[i, 'Details'] = clean_list

In [63]:
#This extracts the dress style description from the item description. 
df["Dress_Style"] = 1
df["Dress_Style"] = df["Dress_Style"].astype('object')
            
for i in range(df.shape[0]):
    if len(df["Details"][i])>0:
        df.at[i, "Dress_Style"] = df["Details"][i][0]

In [64]:
#Clean up the typos in the Dress Feature 
for i in range(df.shape[0]):
    if "hourglass" in str(df["Dress_Style"][i]) or "hpurglass" in str(df["Dress_Style"][i]) or "hourgless" in str(df["Dress_Style"][i]) or "houseglass" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "hourglass"
    elif "wrap" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "wrap"
    elif "maxi" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "maxi"
    elif "blouson" in str(df["Dress_Style"][i]) or "blosuon" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "blouson"  
    elif "shirtdress" in str(df["Dress_Style"][i]) or "shirt dress" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "shirtdress"   
    elif "sheath" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "sheath"   
    elif "a-line" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "a-line"  
    elif "empire" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "empire_waist"
    elif "gown" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "gown"    
    elif "shift" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "shift" 
    elif "slip" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "slip_dress"  
    elif "lining: 80% nylon, 20% spandex" in str(df["Dress_Style"][i]) or "three quarter sleeves" in str(df["Dress_Style"][i])  :
        df.at[i, "Dress_Style"] = "sheath"
    elif "short sleeves with cold shoulder detail" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "maxi"  
    elif "sleeveless with shirt collar" in str(df["Dress_Style"][i]) or "full skirt" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "full skirt" 
    elif "fitted skirt" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "fitted_skirt"
    elif "mermaid" in str(df["Dress_Style"][i]):
        df.at[i, "Dress_Style"] = "mermaid"   
    

In [65]:
df["Dress_Style"]=df["Dress_Style"].replace("houseglass","hourglass", regex=True)


In [66]:
#Condense the Dress_Style Feature to a limited number of values 
list_of_dress_styles = ["hourglass","sheath","shift","gown",
                    "maxi","full skirt","wrap","blouson",
                    "fitted_skirt","empire_waist","a-line"]

for i in range(df.shape[0]):
    if str(df.at[i, "Dress_Style"]) not in list_of_dress_styles:
        df.at[i, "Dress_Style"] = "other"

In [67]:
#These are all the different dress styles 
df.Dress_Style.value_counts()

hourglass       55715
sheath          44857
shift           15024
gown            10795
maxi             7819
full skirt       7028
other            6970
wrap             6132
blouson          2994
fitted_skirt     1839
empire_waist     1264
a-line           1038
Name: Dress_Style, dtype: int64

In [68]:
#Drop the height observation where the height is more than 80 inches. 
df.drop(df[df['Height'] > 80].index, inplace = True)
#Drop the age observation where the weight is more than 80 years olds. 
df.drop(df[df['Age'] > 80].index, inplace = True)
df = df.reset_index()
df.drop(columns="index",inplace=True)

In [69]:
#Body Mass Index, or BMI, is a measurement of body fat based on height and weight.
#It can be calculated using pounds and inches: BMI = (weight in pounds x 703) / (height in inches x height in inches).
df["BMI"] = (df["Weight"]*703)/(df["Height"]*df["Height"])
#Export as an excel file 
df.to_csv('../Data/df_clean.csv', header=True, index=False)

In [70]:
#This is the size of the data set after all the cleaning. 
df.shape

(161329, 23)

In [71]:
#df.drop(columns=["Details",'Product_details'],inplace=True)

In [73]:
#Export as an excel file 
df.to_csv('../Data/df_clean.csv', header=True, index=False)